In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Define paths to your dataset
train_dir = "C:/Users/sajuv/OneDrive/Desktop/datasmall/train"
valid_dir = "C:/Users/sajuv/OneDrive/Desktop/datasmall/valid"
test_dir = "C:/Users/sajuv/OneDrive/Desktop/datasmall/test"

# Define image dimensions and batch size
img_width, img_height = 256, 256
batch_size = 32

# ImageDataGenerator for training set
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# ImageDataGenerator for validation set
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# ImageDataGenerator for test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Define and compile CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(38, activation='softmax')  # Adjust this for the number of classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size)



In [ ]:
# Get predicted labels for test set
Y_pred = model.predict(test_generator)
predicted_labels = np.argmax(Y_pred, axis=1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(train_generator)

# Get the predicted classes for the test data
Y_pred_classes = np.argmax(model.predict(test_generator), axis=-1)

# Get the true classes for the test data
Y_true = test_generator.classes

# Define class names based on unique classes in the dataset
class_names = list(test_generator.class_indices.keys())

# Print the accuracy
print("\nTest Accuracy:", accuracy)

# Print the loss
print("\nTest Loss:", loss)


# Calculate additional metrics
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(Y_true, Y_pred_classes, target_names=class_names))


# Print model summary
print(model.summary())


In [ ]:
# Compute ROC curve and ROC area for each class
from itertools import cycle
from sklearn.metrics import auc, roc_curve


fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(test_generator.class_indices)):
    fpr[i], tpr[i], _ = roc_curve(Y_true == i, Y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute macro-average ROC curve and ROC area
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(test_generator.class_indices))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(test_generator.class_indices)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(test_generator.class_indices)

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Define linewidth
lw = 2

# Get class names
class_names = list(test_generator.class_indices.keys())

# Plot ROC curve
plt.figure(figsize=(15, 12))
# plt.plot(fpr["macro"], tpr["macro"],
#          label='Macro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["macro"]),
#          color='navy', linestyle=':', linewidth=lw)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(len(test_generator.class_indices)), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} ({1}) (area = {2:0.2f})'
             ''.format(i, class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for RESNET Classification')
plt.legend(loc="lower right")
plt.show()


In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(fpr["macro"], tpr["macro"],
         label='Macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=lw)
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for    RESNET Classification')
plt.legend(loc="lower right")
plt.show()